In [1]:
import json
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

/Users/danilobabic/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
folder_path = '/Users/danilobabic/Documents/GitHub/ri_train_data'
words = []
missing_videos = []
video_metadata = []
words_file = '/Users/danilobabic/Documents/GitHub/archive/wlasl_class_list.txt'
missing_videos_file = '/Users/danilobabic/Documents/GitHub/archive/missing.txt'
video_metadata_file = '/Users/danilobabic/Documents/GitHub/archive/WLASL_v0.3.json'
archive_path = '/Users/danilobabic/Documents/GitHub/archive/videos'
mp_holistic = mp.solutions.holistic

In [3]:
def get_words(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip().split('\t')
            words.append(line[1])

In [4]:
get_words(words_file)


In [5]:
def get_missing_files(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            missing_videos.append(line)

In [6]:
get_missing_files(missing_videos_file)

In [7]:
def get_video_metadata(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        return data

In [8]:
video_metadata = get_video_metadata(video_metadata_file)

In [9]:
def create_word_folders(path):
    for word in words:
        folder_path = os.path.join(path, word)
        os.makedirs(folder_path, exist_ok=True)
    

In [10]:
def extract_keypoints(frame, model):         
    results = model.process(frame)                 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
def extract_videos():
    print("USO U FUNKCIJUUUUU")
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        print(video_metadata)
        # for each word we find videos 
        for word in video_metadata:
            for video in word['instances']:
                if video['video_id'] in missing_videos or video['split'] != 'train': 
                    continue
                video_path = os.path.join(folder_path, word['gloss'], video['video_id'])
                os.makedirs(video_path, exist_ok=True)
                cap = cv2.VideoCapture(archive_path + "/" + video['video_id'] + ".mp4")
                print("USO U FUNCKIJU FAJL :" + archive_path + "/" + video['video_id'] + ".mp4")
                if video['frame_end'] == -1:
                    frame_num = 1
                    while True: 
                        ret, frame = cap.read()
                        if not ret:
                            break
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1
                else:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, video['frame_start'])
                    frame_num = video['frame_start']
                    while frame_num <= video['frame_end']:
                        ret, frame = cap.read()
                        if not ret:
                            break 
                        keypoints = extract_keypoints(frame, holistic)
                        npy_path = os.path.join(video_path, str(frame_num))
                        np.save(npy_path, keypoints)
                        frame_num += 1 
                        if frame_num > video['frame_end']:
                            break

In [12]:
#extract_videos()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/07069.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/07068.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/07070.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/07074.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65539.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17710.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17733.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65540.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17711.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17712.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17709.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/17720.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/27208.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65890.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/27214.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65889.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/27215.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/27216.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/27217.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/38544.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/38527.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/38529.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/38530.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/38532.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34826.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34827.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34830.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34831.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34832.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/34839.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36946.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36930.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36931.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36932.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36933.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/36936.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65507.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20980.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20981.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20982.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20983.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20986.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20987.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/65677.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20978.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20992.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/20979.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/69325.mp4
USO U FUNCKIJU F

...

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41812.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41814.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41816.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41896.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41897.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41899.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41900.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41905.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/41907.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/42148.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/42137.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/42138.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45529.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45530.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45532.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45597.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45599.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45601.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45637.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45638.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45639.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45640.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45655.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/45656.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/49890.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/49899.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/49900.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/49902.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50198.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50199.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50201.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50466.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50467.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50469.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50471.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/50472.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56115.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56117.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56119.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56176.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56178.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56180.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56356.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56357.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56359.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/56360.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/58323.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/58325.mp4
USO U FUNCKIJU F

USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61512.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61515.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61590.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61592.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61595.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61699.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61700.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61701.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61702.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61846.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61848.mp4
USO U FUNCKIJU FAJL :/Users/danilobabic/Documents/GitHub/archive/videos/61850.mp4
USO U FUNCKIJU F

In [26]:
def load_data_from_folders():
    frames = []
    labels = []
    word_folders = sorted(os.listdir(folder_path))
    testing_list = ["book","minute","cherry"]
    for word_folder in testing_list:
        word_path = os.path.join(folder_path, word_folder)
        video_folders = sorted([f for f in os.listdir(word_path) if os.path.isdir(os.path.join(word_path, f))])
        for video_folder in video_folders:
            video_path = os.path.join(word_path, video_folder)
            frame_files = sorted(os.listdir(video_path))
            video_frames = []
            for frame_file in frame_files:
                frame_path = os.path.join(video_path, frame_file)
                frame = np.load(frame_path)
                video_frames.append(frame)
            frames.append(video_frames)
            labels.append(label_map[word_folder])
    return frames, labels

In [27]:
label_map = {label:num for num, label in enumerate(words)}

In [28]:
frames, labels = load_data_from_folders()
print(labels)
#y = to_categorical(labels).astype(int)

[0, 0, 0, 0, 0, 418, 418, 418, 418, 418, 418, 1068, 1068, 1068]


In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
input_shape = frames[0].shape  # Update with the shape of your frames
num_classes = len(np.unique(labels))  # Number of unique classes

In [ ]:
model = create_cnn_model(input_shape, num_classes)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(frames, labels, epochs=10, batch_size=32)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)